In [1]:
### 标签平衡器开发
from model import MyFasterRCNN
from data import BedsoreDataModule
from utils import batch2pil, draw_bbox
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch

In [7]:
dm = BedsoreDataModule(root='data', num_valid=0, batch_size=1)
ds = dm.train_dataloader().dataset
len(ds)

1363

In [9]:
all_labels = []
for i in tqdm(ds):
    all_labels.append(i[1]['labels'])

100%|██████████| 1363/1363 [07:51<00:00,  2.89it/s]


In [11]:
all_labels = [i.tolist() for i in all_labels]

In [15]:
coded_labels = torch.zeros(len(all_labels),10)

In [32]:
# 对标签进行n_hot编码
for num,labels in enumerate(all_labels):
    for label in labels:
        coded_labels[num,label] += 1

In [20]:
coded_labels

tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [22]:
coded_labels.sum(0)

tensor([  0.,  63., 258., 178., 432., 504., 179., 428., 196., 191.])

In [25]:
stage_labels = coded_labels[:,1:7]

In [28]:
stage_labels

tensor([[0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        ...,
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0.]])

In [2]:
stage_labesl = torch.load('stage_labesl.pkl')

In [280]:
class Net(torch.nn.Module):
    def __init__(self, num_sample):
        super().__init__()
        self.weights = torch.nn.Parameter(torch.ones(num_sample,1))
    
    def forward(self, x):
        x = sum(self.weights.pow(2) * x,0)
        return x

net = Net(len(stage_labesl))
optimizer = torch.optim.SGD(net.parameters(),lr=1e-3)

In [281]:
for epoch in tqdm(range(600)):
    # 训练
    net.train()
    x = torch.autograd.Variable(stage_labesl)
    out = net(x)
    loss = torch.var(out)
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=10, norm_type=2)
    optimizer.step()
print(loss)

100%|██████████| 600/600 [00:09<00:00, 64.19it/s]

tensor(5761.0659, grad_fn=<VarBackward0>)


In [282]:
net.weights.pow(2)

tensor([[0.9792],
        [0.7318],
        [1.1476],
        ...,
        [1.1476],
        [0.7318],
        [1.1629]], grad_fn=<PowBackward0>)

In [283]:
(stage_labesl * net.weights.pow(2)).sum(0)

tensor([111.9974, 245.9458, 200.6527, 305.5676, 316.4608, 207.9531],
       grad_fn=<SumBackward1>)

In [284]:
torch.max(net.weights.pow(2)),torch.min(net.weights.pow(2)),torch.std(net.weights.pow(2)),torch.mean(net.weights.pow(2))

(tensor(2.7709, grad_fn=<MaxBackward1>),
 tensor(0.2422, grad_fn=<MinBackward1>),
 tensor(0.2779, grad_fn=<StdBackward0>),
 tensor(0.8636, grad_fn=<MeanBackward0>))

In [285]:
torch.max(net.weights.pow(2),0)

torch.return_types.max(
values=tensor([2.7709], grad_fn=<MaxBackward0>),
indices=tensor([233]))

In [287]:
2.7709/0.2422

11.44054500412882